In [110]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

from functions import import_imagedata, label_oh_tf, ImageProcessor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torchvision.models import vgg16
from torchvision.models import resnet101

import cv2

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import random
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import pickle
from fns4wandb import set_lossfn

import keras
from keras.datasets import mnist

import requests
requests.packages.urllib3.disable_warnings()
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [111]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [112]:
model_resnet = resnet101(weights="IMAGENET1K_V1")#.eval

newmodel = torch.nn.Sequential(*(list(model_resnet.children())[:-1]))
newmodel=newmodel.to(device)
print(newmodel)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [113]:
def metrics(label, prediction): #TypeError: Singleton array tensor(3) cannot be considered a valid collection.
    
    label= np.array(label.cpu())

    predictions_np = prediction.cpu().detach().numpy()
    #y_pred' parameter of f1_score must be an array-like or a sparse matrix. Got 7 instead.
    predicted_classes = np.argmax(predictions_np, axis=0)
    #print('metrics Label:   ', label)
    #print('metrics prediction   ', predicted_classes)
    #avg_f1_score = f1_score(label, predictions_np, average='macro')
    acc = accuracy_score(label, predicted_classes)
    
    return acc

In [114]:
def preprocess_im(img):
    IP = ImageProcessor(device='cpu')
    if isinstance(img, str):
        img = cv2.imread(img_path) #
    #imgx = img.shape[0]
    #imgy = img.shape[1]
    #img = img.reshape(imgx, imgy,1)
    #img = IP.blank_padding(img, (224,224))
    img = IP.to_tensor(img).to(device)
    return img


In [115]:
# get len of 

"""file_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'
random_seed =1
img_len = len(os.listdir(file_path))


#print(ids[4])
x, y = import_imagedata(file_path)

x_train, x_test, y_train, y_tests = train_test_split(x,y, test_size=0.2, train_size=0.8,
                                 random_state=random_seed, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, train_size=0.8,
                                 random_state=random_seed, shuffle=True)
"""

"file_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'\nrandom_seed =1\nimg_len = len(os.listdir(file_path))\n\n\n#print(ids[4])\nx, y = import_imagedata(file_path)\n\nx_train, x_test, y_train, y_tests = train_test_split(x,y, test_size=0.2, train_size=0.8,\n                                 random_state=random_seed, shuffle=True)\nx_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, train_size=0.8,\n                                 random_state=random_seed, shuffle=True)\n"

In [116]:
title = f'IDSW_resnet_fine_mnisttest_122023'
save_dict = {'Run' : title,
            'Current_Epoch': 0}
                #r'/its/home/nn268/antvis/optics/
save_location = r'pickles/'#pickles

In [117]:
"""run_title = "IDSW_RESNET_hp_fine_112023"

config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        #'dropout':{
        #    'values': [0.5, 0.4, 0.3]
        #},
        'weight_decay':{
            'values': [1e-5,2e-5, 3e-5,4e-5]
        },
        'epochs':{
            'value': 120
        },
        'lin_layer_size': {
            'values': [100] #, 150, 50
        },
        'first_lin_lay':{
            'values':[248832]
        },
        'optimizer': {
            'values': ['adam']
        },
            'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 1e-6,
                'max': 1e-3
            },
        'loss_fn': {
            'values': ['CrossEntropy'] #'MSE', 
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.2, 0.3, 0.4, 0.6]
        },
        'ks': {
            'values': [(3,5)]
        },
        'dropout':{
            'values': [0.5, 0.4, 0.3]
        },
        'channels':{
            'values': [3]
        },
        'num_classes': {
            'values': [10]
        },
        'model_name' : {'values': ['resnet_mlp']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']
        }
        }
    }

"""

'run_title = "IDSW_RESNET_hp_fine_112023"\n\nconfig = {\n    \'method\': \'random\',\n    \'metric\':{\n        \'goal\': \'minimize\',\n        \'name\': \'val_loss\'},\n    \'parameters\': {\n        #\'dropout\':{\n        #    \'values\': [0.5, 0.4, 0.3]\n        #},\n        \'weight_decay\':{\n            \'values\': [1e-5,2e-5, 3e-5,4e-5]\n        },\n        \'epochs\':{\n            \'value\': 120\n        },\n        \'lin_layer_size\': {\n            \'values\': [100] #, 150, 50\n        },\n        \'first_lin_lay\':{\n            \'values\':[248832]\n        },\n        \'optimizer\': {\n            \'values\': [\'adam\']\n        },\n            \'learning_rate\': {\n                # a flat distribution between 0 and 0.1\n                \'distribution\': \'log_uniform_values\',\n                \'min\': 1e-6,\n                \'max\': 1e-3\n            },\n        \'loss_fn\': {\n            \'values\': [\'CrossEntropy\'] #\'MSE\', \n        },\n        \'data_set\':{\n

In [118]:
save_dict['Run']

'IDSW_resnet_fine_mnisttest_122023'

In [119]:
class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        x = x.squeeze(0)
        x = x.squeeze(1)
        x = x.squeeze(1)
        return x

In [120]:
"""config = dict(
    epochs= 80, #30, 
    learning_rate =1e-6,
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    ks = (3,5),
    scheduler=0.2,
    f_lin_lay = 7168 #1024*7 = 7168
)"""

run_title = "IDSW_resnet_fine_cifar10_122023"

config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        #'dropout':{
        #    'values': [0.5, 0.4, 0.3]
        #},

        'epochs':{
            'value': 2
        },

        'first_lin_lay':{
            'values':[248832]
        },
        'optimizer': {
            'values': ['adam']
        },
            'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 1e-7,
                'max': 1e-2
            },
        'loss_fn': {
            'values': ['CrossEntropy', 'MSE'] #'MSE', 
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.1, 0.01, 0.001]
        },
        'ks': {
            'values': [(3,5)]
        },

        'channels':{
            'values': [3]
        },
        'num_classes': {
            'values': [10]
        },
        'model_name' : {'values': ['resnet_fine']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']
        }
        }
    }



In [121]:
#!wandb login 2bf372d1273e4af99733e10529509d9b252efe88

In [122]:
wandb.login()

True

In [123]:
sweep_id = wandb.sweep(config, project=f"{run_title}")

Create sweep with ID: 366u2jd1
Sweep URL: https://wandb.ai/antvis/IDSW_resnet_fine_cifar10_122023/sweeps/366u2jd1


In [124]:
def plot_grad_flow(model):

    #### This function, used specifically for the visualisation of gradient flow, was found in the following stackoverflow thread: https://stackoverflow.com/questions/70394788/pytorch-adam-optimizer-dramatically-cuts-gradient-flow-compared-to-sgd
    model = model.to('cpu')
    named_parameters = model.named_parameters()
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        #print(n,p)
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            #print('gradddssss', p.grad)
            #print('p: ', p)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=1) #  top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    model = model.to('gpu')

In [125]:
from keras.preprocessing.image import img_to_array, array_to_img
from functions import ImageProcessor

IP= ImageProcessor(device)

def preprocessMNIST():
    #load data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_val,y_train,  y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True)
    #print(x_train.shape,x_test.shape, x_val.shape)
    #print(len(x_train))
    #print('1')
    #IP.view(x_train[0],5)


    def convert(data): # convert greyscale image to a 3d gray image
        data_list = []
        for i in data:
            i = cv2.resize(i, (48,48))
            i = cv2.cvtColor(i, cv2.COLOR_GRAY2BGR)
            #print(type(i))
            data_list.append(i)
        data = np.array(data_list)
        return data


    x_train = convert(x_train)
    x_val = convert(x_val)
    x_test = convert(x_test)
    #print('post convert',x_train[0].shape)
    
    # normalise pixel vals
    x_train = [i/ 255 for i in x_train]
    x_test = [i/ 255 for i in x_test]
    x_val = [i/ 255 for i in x_val]
    
    # ensure array
    x_train= np.array(x_train)
    x_test= np.array(x_test)
    x_val= np.array(x_val)
    #print(type(x_train))
    #print('norm \n', x_train[0].shape, type(x_train),'\n', len(x_train))
    #IP.view(x_train[0],5)

    # convert to float 32
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_val = x_val.astype('float32')
    #print('flaot', x_train.shape, type(x_train), '\n')
    #IP.view(x_train[0],5)
    return x_train,y_train, x_val, y_val,  x_test, y_test

In [134]:


def train_model(model,  x_train, x_val, y_train, y_val, config, best_acc=0): #train_dl, val_dl, 
    #wandb.watch(model, log='all', log_freq=10)
    print(config)
    loss_fn = set_lossfn(config['loss_fn']) # ****
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config['scheduler'], last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
    
    ####
    
    
    model.train()
    
    
    #losses= []
    #predictions = []
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    
    total_epochs = 0
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        print('E   ', epoch)
        t_correct = 0
        v_correct = 0
    
        if epoch == 0:
            model = model.to('cpu')
            best_model = deepcopy(model)
            model = model.to(device)
            
            
        #train_ids = random.shuffle(train_ids)
        #print(type(train_ids))
        print('training...')
        for idx, img in enumerate(x_train): 
            model.train()

            x = preprocess_im(img)

            train_prediction = model.forward(x)
            
            y = label_oh_tf(y_train[idx], device=device, num_classes=10) # use same index val to index y (labels) and turn into onehot encoded label

            """
            if idx % 1000 == 0:
                print(idx, ' / ', len(x_train))
                !nvidia-smi
            """
            #print(train_prediction, train_label)
        
        
            t_loss = loss_fn(train_prediction, y)

            if train_prediction.argmax() == y.argmax():
                t_correct+=1

            
            optimizer.zero_grad()
            t_loss.backward()
            if epoch == config['epochs']-1:
                plot_grad_flow(model)
            
            optimizer.step()
            scheduler.step()
            
            t_loss_list.append(t_loss.to('cpu'))
            t_predict_list.append(train_prediction.to('cpu'))
            t_label_list.append(y.to('cpu'))
            
            train_acc = (t_correct / len(x_train))
        t_accuracy_list.append(train_acc)
            
            # accuracy at every step. every epoch / by x_train
            
        print('validating...')
        
        for idx, img in enumerate(x_val):
            model.eval()
            x = preprocess_im(img)

            val_prediction = model.forward(x)
            
            y = label_oh_tf(y_val[idx], device=device, num_classes=10)
            """
            if idx % 100 == 0:
                print(idx, ' / ', len(x_val))
                !nvidia-smi
            """

            v_loss = loss_fn(val_prediction, y)
            
            if val_prediction.argmax() == y.argmax():
                v_correct +=1
                

            v_loss = v_loss.to('cpu')
            v_loss_list.append(v_loss.item())
            
            
            val_prediction = val_prediction.to('cpu')
            v_predict_list.append(val_prediction.detach().numpy())
            
            v_label_list.append(y.to('cpu').detach().numpy())
            
        val_acc = (v_correct / len(y_val))
        v_accuracy_list.append(val_acc)
            
        total_epochs += epoch

        if val_acc > best_acc:
    
            #print('Start Of SAve -----------------------------')
            #!nvidia-smi

            best_acc = val_acc
            
            model = model.to('cpu')
            best_model = model#deepcopy(model)
            model = model.to(device)
            

            
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['training_samples'] = len(x_train)
            save_dict['validation_samples'] = len(x_val)
            save_dict['t_loss_list'] = t_loss_list
            save_dict['t_predict_list'] = t_predict_list  
            save_dict['t_accuracy_list'] = t_accuracy_list  #
            save_dict['v_loss_list'] = v_loss_list
            save_dict['v_predict_list'] = v_predict_list  #
            save_dict['v_accuracy_list'] = v_accuracy_list  #
            save_dict['t_labels'] = t_label_list
            save_dict['v_labels'] = v_label_list
            
            """model_architecture = [nn.Sequential(
                            model_vgg16,
                            Squeeze(),
                            nn.Linear(4096,10),
                            nn.Softmax(dim=0),
                        )]"""

            save_dict['model.state_dict'] = model.state_dict()# .to('cpu')
            #save_dict['model_architecture_untrained'] = model_architecture

            title = save_dict['Run']
            with open(f'{save_location}{title}.pkl', 'wb+') as f:
                pickle.dump(save_dict, f)
            
            print('improvment in metrics. model saved')

            #print('END Of SAve -----------------------------')
            #!nvidia-smi

        

        if (epoch+1)%2==0:
            train_log(t_loss, v_loss, epoch)
            wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
            wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
            
    model = best_model
    #labels = zip(t_label_list, v_label_list)
    #losses = zip(t_loss_list, v_loss_list)
    #predictions = zip(t_predict_list, v_predict_list)
    return model,save_dict





wandb: WARNING Unable to render progress bar, see the user log for details
wandb: ERROR Problem finishing run
Exception in thread Thread-8 (_run_job):
Traceback (most recent call last):
  File "/its/home/nn268/.local/lib/python3.10/site-packages/tqdm/std.py", line 1192, in __iter__
    self.update(n - last_print_n)
  File "/its/home/nn268/.local/lib/python3.10/site-packages/tqdm/std.py", line 1243, in update
    self.refresh(lock_args=self.lock_args)
  File "/its/home/nn268/.local/lib/python3.10/site-packages/tqdm/std.py", line 1348, in refresh
    self.display()
  File "/its/home/nn268/.local/lib/python3.10/site-packages/tqdm/std.py", line 1495, in display
    self.moveto(pos)
  File "/its/home/nn268/.local/lib/python3.10/site-packages/tqdm/std.py", line 1444, in moveto
    self.fp.write('\n' * n + _term_move_up() * -n)
  File "/its/home/nn268/.local/lib/python3.10/site-packages/tqdm/utils.py", line 195, in inner
    return func(*args, **kwargs)
  File "/its/home/nn268/.local/lib/pyth

In [127]:
"""

x, y, _,_,_,_ = preprocessMNIST()

t = x[0]
print(t.shape, type(t))

#IP = ImageProcessor(device)
#if type(x[5]) == np.ndarray:
#    print('array')
#IP.view(t, 5)
print('4')
#t = np.reshape(t, (48,48,3))
print('t')
plt.imshow(t)
plt.show()"""

"\n\nx, y, _,_,_,_ = preprocessMNIST()\n\nt = x[0]\nprint(t.shape, type(t))\n\n#IP = ImageProcessor(device)\n#if type(x[5]) == np.ndarray:\n#    print('array')\n#IP.view(t, 5)\nprint('4')\n#t = np.reshape(t, (48,48,3))\nprint('t')\nplt.imshow(t)\nplt.show()"

In [128]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x.shape)
        return x
    
    
def pipeline(config): 
    
    resnet = nn.Sequential(
            #nn.Conv2d(1, 64,3),
            newmodel,
            Squeeze(),
            nn.Linear(2048,10), #2048
            nn.Softmax(dim=0),
        )
    
    resnet.to(device)
    model=resnet
    loss_list=[]
    x_train,y_train, x_val, y_val,  x_test, y_test = preprocessMNIST()
    

    model, save_dict = train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl""

    return model, save_dict

"""with wandb.init(project=title, config=config):
    config = wandb.config
    model = resnet

    model, save_dict = train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl""

return model, save_dict"""

'with wandb.init(project=title, config=config):\n    config = wandb.config\n    model = resnet\n\n    model, save_dict = train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl""\n\nreturn model, save_dict'

In [ ]:

#model, save_dict = pipeline(config['loss_fn'])

def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model, save_dict = pipeline(config)
        

wandb.agent(sweep_id, tr, count=20)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [130]:
"""#model,save_dict = pipeline(config) #7,168

def tr(config=None):
    print('1')
    with wandb.init(config=config):
        print('2')
        config = wandb.config
        print('3')
        model, save_dict = pipeline(config)
        
sweep_id = wandb.sweep(config, project=f"{run_title}"),
#wandb.agent(sweep_id, tr, count=20)"""

'#model,save_dict = pipeline(config) #7,168\n\ndef tr(config=None):\n    print(\'1\')\n    with wandb.init(config=config):\n        print(\'2\')\n        config = wandb.config\n        print(\'3\')\n        model, save_dict = pipeline(config)\n        \nsweep_id = wandb.sweep(config, project=f"{run_title}"),\n#wandb.agent(sweep_id, tr, count=20)'

In [131]:
!nvidia-smi


797it [00:25, 12.47it/s]
800it [00:25, 14.22it/s]
803it [00:25, 15.90it/s]
806it [00:26, 17.62it/s]
809it [00:26, 19.83it/s]
812it [00:26, 21.08it/s]
815it [00:26, 22.59it/s]

Thu Dec  7 16:44:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:17:00.0  On |                  Off |
| 50%   63C    P2             235W / 450W |  14432MiB / 24564MiB |     67%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--


818it [00:26, 23.37it/s]
821it [00:26, 23.22it/s]

In [132]:
# confusion matrix


def plot_confusion(predictions:list, actual:list, title:str):
    predict_list = [int(t.argmax()) for t in predictions]
    actual = [int(l.argmax()) for l in actual]

    actual = np.array(actual)
    predict_list = np.array(predict_list)


    #FixedLocator locations (3), usually from a call to set_ticks, does not match the number of labels (11).
    print(f'\n     {title}')
    train_epoch_matrix = confusion_matrix(actual, predict_list, labels= [0,1,2,3,4,5,6,7,8,9,10])
    disp= ConfusionMatrixDisplay(train_epoch_matrix, display_labels=[0,1,2,3,4,5,6,7,8,9,10])
    disp.plot()
    plt.show()


In [133]:
t_predict = save_dict['t_predict_list']
t_labels = save_dict['t_labels']

v_predict = save_dict['v_predict_list'] # WHY IS THERE NOTHING IN V OREDICT LIST!
v_labels = save_dict['v_labels']

plot_confusion(t_predict, t_labels, 'Train Confusion Matrix')
plot_confusion(v_predict, v_labels, 'Validation Confusion Matrix')

KeyError: 't_predict_list'

In [ ]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x)
        return x

In [ ]:
"""model =nn.Sequential(
    PrintLayer(),
    model_vgg16,
    PrintLayer(),
    Squeeze(),
    PrintLayer(),
    nn.Linear(2048,11),
    PrintLayer(),
    nn.Softmax(dim=0),
    PrintLayer()

).to(device)"""

In [ ]:

x = preprocess_im(x_train[0])


train_prediction = model.forward(x)

In [ ]:
model.to('cpu').state_dict()


824it [00:26, 21.47it/s]
827it [00:26, 22.86it/s]
830it [00:27, 23.78it/s]
833it [00:27, 24.38it/s]
836it [00:27, 24.57it/s]
839it [00:27, 24.34it/s]
842it [00:27, 24.95it/s]
845it [00:27, 24.79it/s]
848it [00:27, 25.15it/s]
851it [00:27, 25.76it/s]
854it [00:27, 25.32it/s]
857it [00:28, 25.21it/s]
860it [00:28, 25.13it/s]
863it [00:28, 25.98it/s]
866it [00:28, 26.57it/s]
869it [00:28, 26.33it/s]
872it [00:28, 26.59it/s]
875it [00:28, 26.27it/s]
878it [00:28, 25.64it/s]
881it [00:28, 26.52it/s]
884it [00:29, 26.93it/s]
887it [00:29, 27.15it/s]
890it [00:29, 27.00it/s]
893it [00:29, 26.59it/s]
896it [00:29, 26.40it/s]
899it [00:29, 26.50it/s]
902it [00:29, 26.10it/s]
905it [00:29, 25.08it/s]
908it [00:30, 26.08it/s]
911it [00:30, 26.09it/s]
914it [00:30, 25.81it/s]
917it [00:30, 26.58it/s]
920it [00:30, 27.32it/s]
923it [00:30, 26.79it/s]
926it [00:30, 26.17it/s]
929it [00:30, 25.21it/s]
932it [00:30, 24.38it/s]
935it [00:31, 25.24it/s]
938it [00:31, 25.34it/s]
941it [00:31, 25.88it/s]